The code in this cell is for doing the authentication to Twitter.  It should only be run once in each work session because the generated access token can be reused repeatedly.

This code is based on the example in [this lesson](https://heardlibrary.github.io/digital-scholarship/script/python/authenticate/#getting-an-access-token-using-oauth-2-client-credentials).

If the `requests_oauthlib` module isn't found, open a new terminal window and execute the command:

```
conda install requests-oauthlib
```

Be sure to use a dash and not an underscore.

In [ ]:
from oauthlib.oauth2 import BackendApplicationClient
from requests_oauthlib import OAuth2Session
from pathlib import Path

# This is the format of the twitter_credentials.txt file. clientKey and clientSecret
# are for an application you've created using your developer account. 
# The keys are not real, they are just to show you how the file should be formatted.
# Save file in your current working directory (which may be your home directory).
# To force Python to look in your home directory, uncomment the second 'home = ' command.

'''
clientKey=X09AB1a9upsTOxS0Dd1cA9Ola
clientSecret=662cke38slfibkwofqpsag9t9ej5ja18i
'''

requestTokenUrl = 'https://api.twitter.com/oauth2/token'

home = '.'
#home = str(Path.home()) # uncomment this line to force Python to look in your home directory
credentialsFilename = 'twitter_credentials.txt'
credentialsPath = home + '/' + credentialsFilename
with open(credentialsPath, 'rt') as fileObject:
    lineList = fileObject.read().split('\n')
clientKey = lineList[0].split('=')[1]
clientSecret = lineList[1].split('=')[1]

# you can uncomment the next two lines if you want to check that your keys were retrieved correctly
#print(clientKey)
#print(clientSecret)

clientObject = BackendApplicationClient(client_id=clientKey)
oauth = OAuth2Session(client=clientObject)
accessTokenJson = oauth.fetch_token(token_url=requestTokenUrl, client_id=clientKey, client_secret=clientSecret)
# access token should be cached/saved and used repeatedly rather than making many requests fo a new token
# uncomment the following line to see what the access token is
#print(accessTokenJson)


The code in the following cell actually retrieves data from the Twitter API.  You can re-run it repeatedly for different searches.

In [41]:
import requests
import json
import webbrowser

bearerAccessToken = accessTokenJson['access_token']

# This is based on the Standard Search API instructions at
# https://developer.twitter.com/en/docs/tweets/search/api-reference/get-search-tweets

# These values are for near the Eskind Library
searchLatitude = '36.143132'
searchLongitude = '-86.802463'

radius = '1mi'  # search within a one mile radius

query = input('What do you want to search for? ')

resourceUrl = 'https://api.twitter.com/1.1/search/tweets.json'
paramDict = {'q' : query, 'geocode' : searchLatitude + ',' + searchLongitude + ',' + radius}
r = requests.get(resourceUrl, headers={'Authorization' : 'Bearer '+ bearerAccessToken}, params = paramDict)
data = r.json()
#print(json.dumps(data, indent = 2))

print('People tweeted about that at:')
geocoordinatesList = []
for tweet in data['statuses']:
    if tweet['geo'] != None: # this line prevents the script from crashing when location wasn't given
        latitude = str(tweet['geo']['coordinates'][0])
        longitude = str(tweet['geo']['coordinates'][1])
        print(latitude, longitude)
        geocoordinatesList.append({'latitude': latitude, 'longitude': longitude})
print()

# plot the first coordinates
lat = geocoordinatesList[0]['latitude']
lon = geocoordinatesList[0]['longitude']
zoom = '14' # larger number is zoomed in more
googleMapUrl = 'http://www.google.com/maps/place/'+lat+','+lon+'/@'+lat+','+lon+','+zoom+'z'
print('Here is one place where they tweeted:')
print(googleMapUrl)
status = webbrowser.open_new_tab(googleMapUrl)

What do you want to search for? Vanderbilt
People tweeted about that at:
36.1437001 -86.8010256
36.1426398 -86.8001068
36.1460125 -86.8093215
36.1395125 -86.8040784
36.1460125 -86.8093215
36.1371145 -86.8036337
36.1468919 -86.7991216
36.1457192 -86.8066559
36.1437001 -86.8010256
36.13861 -86.80604
36.1426398 -86.8001068
36.1437001 -86.8010256
36.1371546 -86.804168
36.1457192 -86.8066559

Here is one place where they tweeted:
http://www.google.com/maps/place/36.1437001,-86.8010256/@36.1437001,-86.8010256,14z
